# Validating and Assessing an Existing Model
In this demo we take an existing metadata model, validate that the model meets our requirements, and assess what analytics applications the building supports in its current configuration.

1. **Setup BuildingMOTIF**
    1. Import BuildingMOTIF and associated packages
    1. Load BuildingMOTIF Libraries
    1. Load Medium Office Model
1. **Validate Medium Office Model**
    1. Run validation against loaded Libraries
    1. Read reasons for validation failure
    1. Fix model based on validation report
1. **Find Analytics Applications that the Model supports**
    1. Search G36 library for `Analytics_Application` shapes
    1. Test the model against each shape for support
    1. Read which applications the model supports and what extra is needed for unsupported applications

## 1. Setup BuildingMOTIF
### A. Import BuildingMOTIF and associated packages

In [1]:
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
import os
from buildingmotif.utils import compile_model
from buildingmotif.namespaces import BMOTIF, BRICK, SH, RDFS
from rdflib import Namespace

In [2]:
# Create a buildingMOTIF instance with an in-memory sqlite database
building_motif = BuildingMOTIF("sqlite://")

### B. Load BuildingMOTIF Libraries

In [3]:
brick = Library.load(ontology_graph="../libraries/brick/Brick-subset.ttl")
ashrae_g36 = Library.load(directory="../libraries/ashrae/guideline36/")
manifest = Library.load(directory="mediumOffice-validation/constraints/")

#### Roles of Libraries
- **Brick** - Contains shapes which validate that a model complys with Brick syntax. These shapes will automatically be applied to a model if selected for validation.
- **ASHRAE Guideline 36** - Contains shapes which respresent various section of the Guideline 36 spec. Also contains Templates for directly creating G36 compliant equipment. These shapes are not automatically applied.
- **Manifest** - Contains shapes which provide model-specific constraints. Also selects which Guideline 36 shapes we expect to be satisfied by our model.

### C. Load Medium Office model from file

In [4]:
BLDG = Namespace("http://example.org/building/")
medium_office_model = Model.create(BLDG)
medium_office_model.graph.parse(os.path.join('mediumOffice-validation', 'mediumOffice_brick_compiled.ttl'), format="ttl")

<Graph identifier=68df6de0-3786-4426-88ef-a16903cae497 (<class 'rdflib.graph.Graph'>)>

## 2. Validate Medium Office Model

### A. Validate model against loaded Libraries
The Brick library specifies the rules that need to be satisfied by a correct Brick graph. The Manifest specifies the extact conditions which need to be met by this particular model (3 AHUs with 5 VAVs each that satisfy G36). G36 is needed because it is used by the Manifest.

In [8]:
shape_collections = [brick.get_shape_collection(), manifest.get_shape_collection(), ashrae_g36.get_shape_collection()]
validation_context = medium_office_model.validate(shape_collections)

### B. Read reasons for validation failure

In [9]:
print(f"Model is valid: {validation_context.valid}")

if not validation_context.valid:
    print("Reasons why:")
    for diff in validation_context.diffset:
        print(" -" + diff.reason())

Model is valid: True


### C. Fix model based on validation report
For each validation failure BuildingMOTIF generates a template that would satisfy the failed shape. Any unknown values are exposed as template parameters for the user or an automated process to fill in.

In [7]:
generated_templates = validation_context.as_templates()
for t in generated_templates:
    print('-'*80)
    print(t.body.serialize())
    templ_bindings = {}
    for parameter in t.all_parameters:
        param_value = input(f"Please enter the value for parameter \"{parameter}\":")
        templ_bindings[parameter] = BLDG[param_value]
    graph = t.evaluate(templ_bindings)
    medium_office_model.add_graph(graph)


--------------------------------------------------------------------------------
@prefix P: <urn:___param___#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .

<http://example.org/building/5-Zone-PVAV-2> brick:hasPoint P:p1 .

P:p1 a brick:Supply_Air_Temperature_Setpoint .


Please enter the value for parameter "p1":ASDas
--------------------------------------------------------------------------------
@prefix P: <urn:___param___#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .

<http://example.org/building/5-Zone-PVAV-1> brick:hasPoint P:p2 .

P:p2 a brick:Supply_Air_Temperature_Setpoint .


Please enter the value for parameter "p2":ASdas
--------------------------------------------------------------------------------
@prefix P: <urn:___param___#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .

<http://example.org/building/5-Zone-PVAV> brick:hasPoint P:p3 .

P:p3 a brick:Supply_Air_Temperature_Setpoint .


Please enter the value for parameter "p3":asd

## 3. Find Analytics Applications that the Model supports
### A. Search G36 library for `Analytics_Application` shapes
We select the shapes of type `Analytics_Application` from the ASHRAE Guideline 36 Library. We then test our AHUs against each application. From the output of the test we can see which we support out of the box with our metadata model. We can also look at what we are missing in cases where we don't have the required entities.

In [10]:
shape_collections = [brick.get_shape_collection(), ashrae_g36.get_shape_collection()]
shapes_to_test = ashrae_g36.get_shape_collection().get_shapes_of_definition_type(BMOTIF["Analytics_Application"])


### B. Test the model against each shape for support

In [11]:
conformance = medium_office_model.test_model_against_shapes(
    shape_collections=shape_collections,
    shapes_to_test=shapes_to_test,
    target_class=BRICK["AHU"])


### C. Read which applications the model supports and what extra is needed for unsupported applications

In [12]:
for shape_uri, validation_context in conformance.items():
    _, _, label = next(ashrae_g36.get_shape_collection().graph.triples((shape_uri, RDFS["label"], None)))
    print(f"Fault Condition: {label}")
    print(f"\t Conformance: {validation_context.valid}")
    if not validation_context.valid:
        print("\t Reasons why:")
        for diff in validation_context.diffset:
            print(f"\t- {diff.reason()}")

Fault Condition: MAT too low; should be between OAT and RAT
	 Conformance: True
Fault Condition: MAT too high; should be between OAT and RAT
	 Conformance: True
Fault Condition: Too many changes in Operating State
	 Conformance: False
	 Reasons why:
	- http://example.org/building/5-Zone-PVAV needs between 1 and None instances of https://brickschema.org/schema/Brick#Operating_Mode_Status on path https://brickschema.org/schema/Brick#hasPoint
	- http://example.org/building/5-Zone-PVAV-2 needs between 1 and None instances of https://brickschema.org/schema/Brick#Operating_Mode_Status on path https://brickschema.org/schema/Brick#hasPoint
	- http://example.org/building/5-Zone-PVAV-1 needs between 1 and None instances of https://brickschema.org/schema/Brick#Operating_Mode_Status on path https://brickschema.org/schema/Brick#hasPoint
Fault Condition: SAT too low; should be higher than MAT
	 Conformance: True
Fault Condition: OA fraction is too low or too high; should equal %OA_min
	 Conformance: